In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, models, transforms
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
import os

In [7]:
data = pd.read_csv('C:/Users/29588/Desktop/classify-leaves/train.csv')
print(data.shape)
data.head()

(18353, 2)


,image,label
0,images/0.jpg,maclura_pomifera
1,images/1.jpg,maclura_pomifera
2,images/2.jpg,maclura_pomifera
3,images/3.jpg,maclura_pomifera
4,images/4.jpg,maclura_pomifera


In [8]:
label_mapping = {name: idx for idx, name in enumerate(data['label'].unique())}
data['label'] = data['label'].map(label_mapping)
data.head()

,image,label
0,images/0.jpg,0
1,images/1.jpg,0
2,images/2.jpg,0
3,images/3.jpg,0
4,images/4.jpg,0


In [9]:
train_df, val_df = train_test_split(data, test_size=0.2, stratify=data['label'])

In [13]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

should have our own dataset class, setting up the data pipeline in torch. Responsible for loading the images and corresponding labels, applying transformations, preparing data for the DataLoader.

In [14]:
class LeafDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None):
        self.dataframe = dataframe
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir, self.dataframe.iloc[idx, 0])
        image = Image.open(img_name).convert('RGB')
        label = self.dataframe.iloc[idx, 1]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label


In [15]:
img_dir = 'C:/Users/29588/Desktop/classify-leaves'  # Update this if images are in a different directory
train_dataset = LeafDataset(dataframe=train_df, img_dir=img_dir, transform=transform)
val_dataset = LeafDataset(dataframe=val_df, img_dir=img_dir, transform=transform)

In [25]:
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [26]:
model = models.vgg16(pretrained=True)

c:\Users\29588\anaconda3\envs\d2l\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\29588\anaconda3\envs\d2l\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\29588/.cache\torch\hub\checkpoints\vgg16-397923af.pth
100.0%


In [39]:
for param in model.parameters():
    param.requires_grad = False

(Linear(in_features=4096, out_features=176, bias=True),
 Linear(in_features=4096, out_features=4096, bias=True),
 Linear(in_features=25088, out_features=4096, bias=True))

In [30]:
num_classes = len(label_mapping)
model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)

In [31]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [32]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.0001)

In [33]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}")

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
    
    val_acc = correct / len(val_dataset)
    print(f"Validation Accuracy: {val_acc:.4f}")

Epoch [1/10], Loss: 3.9776503250230113
Validation Accuracy: 0.4132
Epoch [2/10], Loss: 2.725160129189751
Validation Accuracy: 0.5489
Epoch [3/10], Loss: 2.218575934179468
Validation Accuracy: 0.6083
Epoch [4/10], Loss: 1.91521705988965
Validation Accuracy: 0.6448
Epoch [5/10], Loss: 1.7016681165217313
Validation Accuracy: 0.6723
Epoch [6/10], Loss: 1.5514554874829478
Validation Accuracy: 0.6878
Epoch [7/10], Loss: 1.4252493139965083
Validation Accuracy: 0.7014
Epoch [8/10], Loss: 1.32748865743608
Validation Accuracy: 0.7153
Epoch [9/10], Loss: 1.254329069767123
Validation Accuracy: 0.7290
Epoch [10/10], Loss: 1.182167007902349
Validation Accuracy: 0.7409


In [40]:
test_data = pd.read_csv('C:/Users/29588/Desktop/classify-leaves/test.csv')

In [43]:
class TestDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None):
        self.dataframe = dataframe
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir, self.dataframe.iloc[idx, 0])
        image = Image.open(img_name).convert('RGB')
        
        if self.transform:
            image = self.transform(image)

        return image, self.dataframe.iloc[idx, 0]  # Return image and path

In [41]:
torch.save(model.state_dict(), 'leaf_classification_vgg16.pth')

In [44]:
test_img_dir = 'C:/Users/29588/Desktop/classify-leaves'  # Adjust as needed
test_dataset = TestDataset(dataframe=test_data, img_dir=test_img_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [45]:
predictions = []
image_paths = []

model = model.to('cuda')
model.eval()

print('device: ', device)

with torch.no_grad():
    for images, paths in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.cpu().numpy())
        image_paths.extend(paths)

device:  cuda


In [46]:
inverse_label_mapping = {v: k for k, v in label_mapping.items()}  # Invert the label mapping
predicted_labels = [inverse_label_mapping[pred] for pred in predictions]

In [47]:
test_data['label'] = predicted_labels

In [49]:
test_data.to_csv('C:/Users/29588/Desktop/classify-leaves/tmp/test_with_pred.csv', index=False)